In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
import tensorflow as tf
from keras.layers import Dense, Input

In [2]:
# Read the data
df = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

# get X, y from the training data
X = df.drop(['id', 'defects'], axis=1).values
y = df.defects.astype('int').values

In [3]:
# Use sklearn KFold cross validation
stratified_cv = StratifiedKFold(n_splits=10)

In [8]:
# Apply cross_val_score on the created model using metric "roc_auc"

n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
accuracies = []
for train_index, val_index in kf.split(X, y):
    X_fold, X_val_fold = X[train_index], X[val_index]
    y_fold, y_val_fold = y[train_index], y[val_index]

    # Create and train your neural network model
    model = tf.keras.Sequential([
        Input(shape=(X_fold.shape[1],)),  # Input layer with input_dim features
        Dense(8, activation='relu'),
        Dense(4, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer with sigmoid activation to get output in range [0, 1]
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')  # Adjust loss and optimizer as needed
    history = model.fit(X_fold, y_fold, epochs=50, batch_size=32,
                    validation_data=(X_val_fold, y_val_fold),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

    # Evaluate the model on the validation data
    y_pred = model.predict(X_val_fold)
    accuracy = roc_auc_score(y_val_fold, y_pred)
    accuracies.append(accuracy)
    print(f'Accuracy on validation fold: {accuracy}')

# After iterating through all folds, analyze the results
print(accuracies)

Epoch 1/50
2545/2545 [==============================] - 3s 997us/step - loss: 2.1964 - val_loss: 0.7303
Epoch 2/50
2545/2545 [==============================] - 2s 958us/step - loss: 1.5692 - val_loss: 2.3877
Epoch 3/50
2545/2545 [==============================] - 2s 966us/step - loss: 0.8422 - val_loss: 1.1659
Epoch 4/50
2545/2545 [==============================] - 2s 963us/step - loss: 0.8193 - val_loss: 0.5155
Epoch 5/50
2545/2545 [==============================] - 2s 974us/step - loss: 0.5952 - val_loss: 0.8398
Epoch 6/50
2545/2545 [==============================] - 2s 975us/step - loss: 0.6129 - val_loss: 0.8900
Epoch 7/50
2545/2545 [==============================] - 2s 964us/step - loss: 0.5692 - val_loss: 0.5353
Epoch 8/50
2545/2545 [==============================] - 2s 980us/step - loss: 0.5175 - val_loss: 0.9100
Epoch 9/50
2545/2545 [==============================] - 2s 973us/step - loss: 0.5810 - val_loss: 1.0669
Epoch 10/50
2545/2545 [==============================] - 2s 980u

In [6]:
# Try different Networks and see how the score will change